# Data Science
- Student: Dmytro Geleshko
- Group: IP-91
- Var: 6

# Imports

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

# Config

In [18]:
# var 6
np.random.seed(6)
plt.rcParams["figure.figsize"] = (20, 8)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.3f}".format

# Load data

In [19]:
df = pd.read_csv("mod_data.csv", index_col="date", parse_dates=True, dtype=np.float32)
df.drop(columns=["close", "volume", "log_close", "log_fit"], inplace=True)
df

,volume_norm,log_diff,day_sin,day_cos,year_sin,year_cos
date,,,,,,
2010-07-18,-0.674,-0.878,-0.000,1.000,-0.266,-0.964
2010-07-19,-0.672,-0.884,-0.000,1.000,-0.283,-0.959
2010-07-20,-0.673,-0.890,-0.000,1.000,-0.299,-0.954
2010-07-21,-0.672,-0.896,0.000,1.000,-0.316,-0.949
2010-07-22,-0.666,-0.902,-0.000,1.000,-0.332,-0.943
...,...,...,...,...,...,...
2021-11-21,-0.523,1.564,-0.000,1.000,-0.643,0.766
2021-11-22,-0.392,1.523,-0.000,1.000,-0.630,0.777
2021-11-23,-0.406,1.545,-0.000,1.000,-0.616,0.787


In [20]:
df.index.dtype, df.dtypes

(dtype('<M8[ns]'),
 volume_norm    float32
 log_diff       float32
 day_sin        float32
 day_cos        float32
 year_sin       float32
 year_cos       float32
 dtype: object)

In [21]:
log_fit_coef = np.array([[ 9.41599146e+00], [-5.06317303e-08]], dtype=np.float64)

## Test

In [23]:
def create_model(Tx, features, Ty):
    X_in = layers.Input(shape=(Tx, features))

    X = layers.LSTM(42, activation="relu", return_sequences=False)(X_in)
    
    X_out = layers.Dense(7, activation="relu")(X)

    return tf.keras.Model(inputs=X_in, outputs=X_out)

In [22]:
X_features = ["volume_norm", "log_diff", "day_sin", "day_cos", "year_sin", "year_cos"]

In [24]:
# input: past 3 weeks
Tx = 21
# output: next 1 week
Ty = 7

model = create_model(Tx, len(X_features), Ty)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 21, 6)]           0         
                                                                 
 lstm_1 (LSTM)               (None, 42)                8232      
                                                                 
 dense_1 (Dense)             (None, 7)                 301       
                                                                 
Total params: 8,533
Trainable params: 8,533
Non-trainable params: 0
_________________________________________________________________


In [38]:
# X is (m, Tx, n)
# y is (m, Ty)
# where m - data size, Tx - input width, n - features, Ty - output width
 
X = np.zeros((df.shape[0] - (Tx + Ty), Tx, len(X_features)))
y = np.zeros((df.shape[0] - (Tx + Ty), Ty))

for i in range(y.shape[0]):
    X[i, :, :] = df[X_features].iloc[i:i+Tx]
    y[i, :] = df["log_diff"].iloc[i+Tx: i+Tx+Ty]

X.shape, y.shape

((4121, 21, 6), (4121, 7))

In [40]:
train_size = int(X.shape[0] * 0.9)

X_train = X[:train_size, :, :]
y_train = y[:train_size, :]

X_test = X[train_size:, :, :]
y_test = y[train_size:, :]

print("X_train", X_train.shape)
print("y_train", y_train.shape)

print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (3708, 21, 6)
y_train (3708, 7)
X_test (413, 21, 6)
y_test (413, 7)


In [25]:
opt = tf.optimizers.Adam(learning_rate=0.01, decay=0.01)
model.compile(optimizer=opt, loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])

E:\Programs\Anaconda\envs\tf2.7\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
